In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from transformers import AutoTokenizer, OpenAIGPTForSequenceClassification, Trainer, TrainingArguments
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

In [3]:
torch.cuda.is_available()

False

In [4]:
train_dataset = pd.read_csv('datasets/train_level_a.csv')
train_dataset

,id,tweet,subtask_a
0,86426,@USER She should ask a few native Americans wh...,OFF
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,16820,Amazon is investigating Chinese employees who ...,NOT
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT
...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT
13237,82921,@USER And why report this garbage. We don't g...,OFF
13238,27429,@USER Pussy,OFF


In [5]:
le = LabelEncoder()

train_texts = train_dataset["tweet"].tolist()
train_labels = le.fit_transform(train_dataset["subtask_a"]).tolist()

In [6]:
tokenizer = AutoTokenizer.from_pretrained("openai-gpt")
model = OpenAIGPTForSequenceClassification.from_pretrained("openai-gpt")

Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    
model.config.pad_token_id = tokenizer.pad_token_id

Using pad_token, but it is not set yet.


In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset = CustomDataset(train_encodings, train_labels)

In [13]:
training_args = TrainingArguments(
    output_dir="models/GPT",
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
trainer.train()

c:\users\bjanka\appdata\local\programs\python\python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13240
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1655
  Number of trainable parameters = 116537088


Step,Training Loss
500,0.539400
1000,0.469400
1500,0.452100


Saving model checkpoint to models/GPT\checkpoint-500
Configuration saved in models/GPT\checkpoint-500\config.json
Model weights saved in models/GPT\checkpoint-500\pytorch_model.bin
Saving model checkpoint to models/GPT\checkpoint-1000
Configuration saved in models/GPT\checkpoint-1000\config.json
Model weights saved in models/GPT\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to models/GPT\checkpoint-1500
Configuration saved in models/GPT\checkpoint-1500\config.json
Model weights saved in models/GPT\checkpoint-1500\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1655, training_loss=0.4860759815780778, metrics={'train_runtime': 11833.13, 'train_samples_per_second': 1.119, 'train_steps_per_second': 0.14, 'total_flos': 1148664268800000.0, 'train_loss': 0.4860759815780778, 'epoch': 1.0})

In [15]:
test_dataset = pd.read_csv('datasets/test_level_a.csv')
test_dataset

,id,tweet,label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,13876,#Watching #Boomer getting the news that she is...,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,OFF
...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,OFF
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT
857,67018,3 people just unfollowed me for talking about ...,OFF
858,50665,#WednesdayWisdom Antifa calls the right fascis...,NOT


In [18]:
test_texts = test_dataset["tweet"].tolist()
test_labels = le.fit_transform(test_dataset["label"]).tolist()

In [20]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'NOT': 0, 'OFF': 1}


In [21]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [26]:
test_dataset = CustomDataset(test_encodings, test_labels)

In [29]:
predictions = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 860
  Batch size = 8


In [37]:
y_pred = np.argmax(predictions[0], axis=1)
y_true = test_labels

In [38]:
print(accuracy_score(y_true, y_pred))
print()
print(classification_report(y_true, y_pred, target_names=le.classes_))

0.8337209302325581

              precision    recall  f1-score   support

         NOT       0.87      0.90      0.89       620
         OFF       0.73      0.65      0.69       240

    accuracy                           0.83       860
   macro avg       0.80      0.78      0.79       860
weighted avg       0.83      0.83      0.83       860

